In [3]:
import numpy as np
import cv2

template = cv2.imread('Template-4.png')        #ดึงรูป
hsv_frame = cv2.cvtColor(template, cv2.COLOR_BGR2HSV)    #การแปลงรูปจาก BGR เป็น HSV
template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)   #การแปลงรูปจาก BGR เป็น gray
avg_color_per_row = np.average(hsv_frame, axis=0)  
avg_color_template = np.average(avg_color_per_row, axis=0)  #เป็นการหาค่าเฉลี่ยสีของภาพ template


Color1 = np.array([int(avg_color_template[0]),int(avg_color_template[1]+10),30]) #เป็นการเอาค่าเฉลี่ยของสีที่หาได้มาทำเป็นค่า min
Color2 = np.array([100,255,255]) #เป็นการกำหนดค่าสี max


IP_file = 'left_output-1.avi' #เอาที่อยู่วิดีโอมาเก็บไว้ในตัวแปร
vid_file = cv2.VideoCapture(IP_file)

while True:
    read_successful, frame = vid_file.read() #เอาวิดีโอมาอ่าน
    query_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #แปลงภาพเฟรมให้เป็นสี gray
    if read_successful: #เช็คสถานะของ read_successful
        sift = cv2.SIFT_create() # SIFT_create เป็นการหาจุดที่คล้ายระหว่างภาพกับวิดีโอ เลือกใช้งานชิบในการตรวจจับ
        bf = cv2.BFMatcher() #ตัวตรวจจับเอาไว้ตรวจจับคุณสมบัติของภาพ
        template_kpts, template_desc = sift.detectAndCompute(template_gray, None) #detectAndCompute เป็นการหา คีย์พอทย์(จุดระหว่างภาพ) กับ ดีสคริปชั่น(ตำแหน่งของคีย์พอทย์)

        dst3 = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  #เป็นการแปลงภาพ frame จาก RGB to BGR
        hsv = cv2.cvtColor(dst3, cv2.COLOR_BGR2HSV) #เป็นการแปลงภาพ dst3 เป็น HSV
        
        mask = cv2.inRange(hsv, Color1, Color2) #เป็นการสร้างมาร์ค
        # cv2.imshow("imim1",mask)
        
        #เป็นการสร้าง kernel
        kernel2 = np.ones((3,3),np.uint8)
        kernel3 = np.ones((4,4),np.uint8)
        kernel4 = np.ones((5,5),np.uint8)
        kernel5 = np.ones((7,7),np.uint8)
        
        opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel2) #เป็นการทำให้ภาพตรงกลางโปร่งแสง
        dilation = cv2.dilate(opening,kernel5,iterations = 2) #เป็นการกำหนดขอบของภาพ
        
        erosion = cv2.erode(dilation,kernel4,iterations = 2) #เป็นการทำให้ภาพเรียว
        opening2 = cv2.morphologyEx(erosion, cv2.MORPH_OPEN, kernel2) #เป็นการทำให้ภาพตรงกลางโปร่งแสง
        erosion2 = cv2.erode(opening2,kernel5,iterations = 2) #เป็นการทำให้ภาพเรียว
        opening3 = cv2.morphologyEx(erosion2, cv2.MORPH_OPEN, kernel5) #เป็นการทำให้ภาพตรงกลางโปร่งแสง
        
        dilation2 = cv2.dilate(opening3,kernel3,iterations = 1) #เป็นการกำหนดขอบของภาพ
        # cv2.imshow("imim2",dilation2)
        
        contours, hier = cv2.findContours(dilation2,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE) #เป็นการหามุมของภาพ
        w_max = frame.shape[1] / 2 #เอาความกว้างของ frame มาหาร 2
        h_max = frame.shape[0] / 2 #เอาความสูงของ frame มาหาร 2
        
        for cnt in contours:
            Z = cv2.contourArea(cnt) #หาพื้นที่ภายใน contours เอามาเก็บไว้ในตัวแปร Z
            if 1350<Z: #ถ้า Z มากกว่า 1350 ให้ทำงาน
                (x,y,w,h) = cv2.boundingRect(cnt) #เป็นการสร้างกรอบสี่เหลี่ยมมา แล้วเก็บค่า x,y,w,h ของกรอบสี่เหลี่ยม

                
                x1 = x-30 
                y1 = y-30-int(w*0.55)
                
                
                cropped_image = query_gray[y1:y+h+30 , x1:x+w+30] #จับเฉพาะภาพของหนังสือ
                # cv2.imshow("imim22",cropped_image)
                query_kpts, query_desc = sift.detectAndCompute(cropped_image, None) #detectAndCompute เป็นการหา คีย์พอทย์(จุดระหว่างภาพ) กับ ดีสคริปชั่น(ตำแหน่งของคีย์พอทย์)

                matches = bf.knnMatch(template_desc, query_desc, k=2) #เป็นการจับคู่แมทช์จากตำแหน่งของคีพอทย์
        
                good_matches = list() #สร้าง list เก็บไว้ใน good_matches
                for m, n in matches : #วนลูปเพื่อหาคู่แมทช์ที่ดีที่สุด
                    if m.distance < 0.75*n.distance : 
                        good_matches.append(m)

                if len(good_matches) > 7 : #เป็นการจับภาพ template และภาพ frame ที่เหมือนกัน 
                    src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
                    dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

                    H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 1.0) 
                    h2, w2 = template.shape[:2]
                    
                    template_box = np.float32([[0, 0], [0, h2-1], [w2-1, h2-1], [w2-1, 0]]).reshape(-1,1,2)
                    transformed_box = cv2.perspectiveTransform(template_box, H)
                    # print(transformed_box)

                    detected_img = cv2.polylines(frame[y1:y+h+30 , x1:x+w+30], [np.int32(transformed_box)], True, (0,255,0), 3, cv2.LINE_AA) 
                    frame[y1:y+h+30 , x1:x+w+30] = detected_img 
    else:
        break
    cv2.imshow('Detect book',frame) 
    key = cv2.waitKey(15)
    if key == ord('q'):
        break
vid_file.release()
cv2.destroyAllWindows()